**USE MAPPING ENVIRONMENT**

## Ruapehu 2022 January seismicity for VAB

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, Point

import datetime as dt
import numpy as np

import cartopy.crs as ccrs
from osgeo import gdal, osr
gdal.UseExceptions()

In [ ]:
def findbounds(infile, ds):
    xpx = ds.RasterXSize
    ypx = ds.RasterYSize
    
    tfw = open(infile, 'r')
    content = [line.strip() for line in tfw.readlines()]
    left = float(content[4])
    top = float(content[5])
    right = float(content[4]) + xpx * float(content[0])
    bottom = float(content[5]) + ypx * float(content[3])
    return (left, right, bottom, top)

### Acquire earthquake data from GeoNet - Want to archive this and use a 'static' copy so data can't change if database changes
Data acquired from GeoNet 2022-01-04 UTC.

In [ ]:
# curl "http://wfs.geonet.org.nz/geonet/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=geonet:quake_search_v1&maxFeatures=1000&outputFormat=csv&cql_filter=depth<10+AND+DWITHIN(origin_geom,Point+(175.564+-39.281),5000,meters)+AND+origintime>='2019-01-01'" -o ruapehu_5km_since_2019.csv

# start = '1980-01-01T00:00:00'
# end = '2022-01-01T00:00:00'

#box for search area, this is plenty big enough to at least get regional event distribution
# bbox = '175.4,-38.9,176.4,-38.2'
# maxdep = '30'

# url = 'https://quakesearch.geonet.org.nz/csv?bbox='+bbox+'&maxdepth='+maxdep+'&startdate='+start+'&enddate='+end

# df = pd.read_csv(url)

# df.to_csv('geonet_quakesearch_1980-2021.csv', index=False)

In [ ]:
#read previously acquired data file
df = pd.read_csv('ruapehu_5km_since_2019.csv', parse_dates=['origintime'], usecols=[2,4,5,6,7,9,10,13])

In [ ]:
df.head()

In [ ]:
#eqs as a geopandas dataframe
eqs = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs='EPSG:4326')
#compatible crs
eqs.to_crs(crs=2193, inplace=True)

### Map all epicentres

In [ ]:
base = './tiff/nz-topo50-maps'
fname = base+'.tif'
tfwfile = base+'.tfw'
ds = gdal.Open(fname)
data = ds.ReadAsArray()

extent = findbounds(tfwfile, ds)

projection = ccrs.epsg('2193') #for NZTM

In [ ]:
extent

In [ ]:
#find only earthquakes within area of map
#geodataframe covering map area
# ma = gpd.GeoSeries(Polygon([(extent[0], extent[2]), (extent[0], extent[3]), (extent[1], extent[3]), (extent[1], extent[2])]), crs=2193)
# map_area = gpd.GeoDataFrame(geometry=gpd.GeoSeries(ma))

#intersection
# eqsmap = gpd.overlay(eqs, map_area, how='intersection')

In [ ]:
#number of earthquakes selected
len(eqs)

In [ ]:
eqs.head()

In [ ]:
eqsrecent = eqs[eqs['origintime']>='2021-12-30']
eqsold = eqs[eqs['origintime']<'2021-12-30']

In [ ]:
len(eqsrecent)

### 5 km circle

In [ ]:
rcl = gpd.GeoSeries(Point(175.564,-39.281), crs=4326)
vent = rcl.to_crs(crs=2193)
vent

In [ ]:
circle = vent.buffer(5000)

## Time series analysis

In [ ]:
#date column from origin time, uses first of month for every date in that month
eqs['date'] = pd.to_datetime(eqs['origintime'].dt.year.astype(str) + eqs['origintime'].dt.month.astype(str), format='%Y%m')

In [ ]:
eqs.head()

In [ ]:
#group by month
month = eqs.groupby(['date']).agg({'magnitude': 'count'})
month.rename(columns={'magnitude': 'count'}, inplace=True)

In [ ]:
fig,ax1 = plt.subplots(1, 1, figsize=(10,5))

ax1.vlines(month.index, 0, month['count'], color='cornflowerblue', linewidth=10)
# ax.xaxis.grid()
# ax.yaxis.grid()

ax1.set_xlim(left=dt.datetime(2019,1,1))
ax1.set_ylim(bottom=0)
ax1.set_ylabel('Earthquakes per month', fontsize=12)
ax1.set_title('Monthly number', fontsize=16)
# plt.savefig('geonet_time-series_monthly_number_distance.png', facecolor='white', dpi=200)

## Map with time-series superimposed

In [ ]:
subplot_kw = dict(projection=projection)
fig,ax = plt.subplots(subplot_kw=subplot_kw, dpi=250)
img = ax.imshow(data[:3, :, :].transpose((1, 2, 0)), extent=extent, origin='upper', alpha=0.7)

#eqs epicentres as circles
eqsold.plot(ax=ax, marker='o', markersize=1.5, color='red', alpha=0.2, label='2019 to 2021 Dec 29')
eqsrecent.plot(ax=ax, marker='o', markersize=1.5, color='black', label='since 2021 Dec 30')

#5 km radius circle
circle.plot(ax=ax, facecolor='none', edgecolor='cornflowerblue', linestyle='--')

ax.legend(loc='upper left', fontsize=4)

ax.text(0.58, 0.03,'Data Souce: GeoNet\nwww.geonet.org.nz/data/types/eq_catalogue',
        fontsize=4,
        color='grey',
        bbox=dict(facecolor='white', boxstyle='round', edgecolor='grey'),
        horizontalalignment='left',
        verticalalignment='bottom',
        transform = ax.transAxes)

ax.text(0.58, 0.1,'Basemap: LINZ\nwww.linz.govt.nz',
        fontsize=4,
        color='grey',
        bbox=dict(facecolor='white', boxstyle='round', edgecolor='grey'),
        horizontalalignment='left',
        verticalalignment='bottom',
        transform = ax.transAxes)

plt.suptitle('Recent earthquakes cluster near Ruapehu\'s crater lake', fontsize=8, y=0.95)
ax.set_title('Earthquakes since 2019 within 5 km of crater and less than 10 km deep', fontsize=5)

# this is an inset axes over the main axes
# position/size doesn't work the way I expect, adjusted till looked okay
ax1 = fig.add_axes([.28, .17, .25, .125], facecolor='white')

ax1.vlines(month.index, 0, month['count'], color='cornflowerblue', linewidth=2, label='monthly number')
ax1.set_xlim(left=dt.datetime(2019,1,1))
ax1.set_ylim(bottom=0)

plt.yticks([0,10,20, 30], fontsize=4)
ax1.yaxis.set_tick_params(pad=1)

#x is messy
x = ['2019-01-01', '2020-01-01', '2021-01-01', '2022-01-10']
xdates = [dt.datetime.strptime(dstr,'%Y-%m-%d') for dstr in x]
plt.xticks(xdates, fontsize=4)
ax1.xaxis.set_tick_params(pad=1)

ax1.legend(loc='upper left', fontsize=4)

plt.savefig('map_revised.png', facecolor='white', dpi=600, bbox_inches='tight')